In [1]:
import os
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("C:/Defi_IA/data/Dataset/weather.csv")
df.head()

,station,y,test,month,day,latitude,longitude,height,mean_ff_by_station_day,mean_t_by_station_day,mean_td_by_station_day,mean_hu_by_station_day,mean_dd_by_station_day,mean_precip_by_station_day
0,14066001,3.4,False,1,0,49.334,-0.431,2.0,3.641875,279.904167,278.099167,88.858333,176.625000,0.008333
1,14126001,0.5,False,1,0,49.145,0.042,125.0,NaN,279.723958,278.852083,94.302083,NaN,0.020833
2,14137001,3.4,False,1,0,49.180,-0.456,67.0,4.416458,279.648125,278.187500,90.820833,170.916667,0.004255
3,14216001,4.0,False,1,0,48.928,-0.149,155.0,3.727292,279.662500,277.976250,89.404167,186.125000,0.012500
4,14296001,13.3,False,1,0,48.795,-1.037,336.0,NaN,278.587917,NaN,NaN,NaN,0.004167


In [7]:
df[df.y.isna()]

,station,y,test,month,day,latitude,longitude,height,mean_ff_by_station_day,mean_t_by_station_day,mean_td_by_station_day,mean_hu_by_station_day,mean_dd_by_station_day,mean_precip_by_station_day
9,14515001,NaN,False,1,0,49.349,-0.773,68.0,4.140417,NaN,NaN,NaN,187.791667,NaN
38,28134004,NaN,False,1,0,48.726,1.387,139.0,NaN,279.568101,252.944477,47.780390,NaN,NaN
40,28198001,NaN,False,1,0,48.061,1.376,126.0,2.522708,278.804421,277.934167,94.451852,168.687500,0.021277
46,28280003,NaN,False,1,0,48.314,0.839,180.0,NaN,279.026763,278.445497,96.154729,NaN,NaN
52,29022001,NaN,False,1,0,48.279,-4.624,52.0,11.561250,NaN,NaN,NaN,183.041667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268882,86137003,NaN,True,12,293,47.035,0.098,96.0,1.979792,278.563125,276.517917,88.504167,199.541667,0.008333
268883,86165005,NaN,True,12,293,46.412,0.841,153.0,1.412470,278.652083,276.493958,87.447917,181.223214,0.008333
268884,86273001,NaN,True,12,293,46.464,1.042,121.0,NaN,274.948750,NaN,NaN,NaN,0.008333
268885,91200002,NaN,True,12,293,48.526,1.993,116.0,1.161250,278.336875,276.546875,88.891667,179.250000,0.004167


In [ ]:
# The "test" column is a boolean column which is equal to True for test and False for train.
X_train = df[df["test"] == False].copy()
X_test = df[df["test"]].copy()

X_train = X_train[X_train["y"].notnull()]

# y_train is the target variable to train our model.
y_train = X_train["y"].astype(float).copy()

X_test["id"] = X_test["station"].astype(str) + "_" + X_test["day"].astype(str)
X_test = X_test.set_index("id")

columns_to_drop = ["y", "test"]

X_train = X_train.drop(columns_to_drop, axis = "columns")
X_test = X_test.drop(columns_to_drop, axis = "columns")

In [ ]:
X_train.head()

In [ ]:
df_train = df[df.test==False].fillna(-1)
nb_day_station = df_train.station.value_counts()
df_train = df_train[df_train.station.isin(nb_day_station[nb_day_station == 730].index)]

y_train = df_train.y.astype(float).copy()
columns_to_drop = ["y", "test"]
X_train = df_train.drop(columns_to_drop, axis = "columns")
X_train = np.array(list(X_train.groupby('station').apply(np.array)))

y_train = np.array(list(df_train.groupby('station').y.apply(np.array)))

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

def lstm_model():
    model = Sequential()
    model.add(LSTM(32, input_shape=X_train.shape[1:], activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics='mean_absolute_percentage_error')
    
    return model

lstm_model().summary()

In [ ]:
model = lstm_model()
model.fit(X_train, y_train, epochs=2, batch_size=10, verbose=2)